Information accurate as of September 22nd, 2023 @ 12am

In [1]:
import json
import pathlib
import math
import os
from typing import Union

import httpx
import pandas as pd
from bs4 import BeautifulSoup
from pydantic import BaseModel
from tqdm.notebook import tqdm

In [2]:
base_url: str = "https://huggingface.co"

In [3]:
data = httpx.request(method="GET", url=base_url + '/models').text
soup = BeautifulSoup(data, "html.parser")

In [4]:
tasks: dict = {}
# Find all tasks in huggingface/models menu
for a in soup.find_all(name="a", class_="tag tag-white"):
    a: BeautifulSoup = a
    # Get each task name and href tag
    tasks[str(a.text).strip().replace("\n", "")] = {
        "link": a.get("href").replace("models", "models-json")
    }

In [5]:
class Model(BaseModel):
    downloads: int
    id: str
    lastModified: str
    likes: int
    pipeline_tag: str
    private: bool
    repoType: str
    author: str or None = None
    authorData: dict or None = None


class TaskResponse(BaseModel):
    activeFilters: dict
    models: list
    numItemsPerPage: int
    numTotalItems: int
    numPages: Union[int, None] = None
    pageIndex: int

In [6]:
def build_url(
    base: str, page: Union[int, None] = None, sort: Union[str, None] = None
) -> str:
    """Helper function to build huggingface urls."""
    page = f"&p={page}" if page else ""
    sort = (
        f"&sort={sort}"
        if (sort and sort in ["trending", "likes", "downloads", "modified"])
        else ""
    )
    return base + page + sort

In [8]:
# Visit each base level task link and determine total number of pages per task
for key, task in tqdm(tasks.items()):
    task_url: str = build_url(base_url + task["link"], sort="downloads")
    print(f"requesting: {task_url}")
    # Return HTTP response as json and parse using pydantic validator
    resp: TaskResponse = TaskResponse(
        **httpx.request(method="GET", url=task_url).json()
    )
    # Calculate total number of pages and always round up
    resp.numPages = math.ceil(resp.numTotalItems / resp.numItemsPerPage)
    # Extract model information on page 1
    tasks[key]["models"] = [Model(**model) for model in resp.models]
    # Iterate through all pages. Build new url to visit using iterator index.
    for page in tqdm(range(resp.numPages)):
        sub_page_url: str = build_url(
            base=base_url + task["link"], page=page + 1, sort="downloads"
        )
        print(f"requesting: {sub_page_url}")
        sub_page_resp: TaskResponse = TaskResponse(
            **httpx.request(method="GET", url=sub_page_url).json()
        )
        # Extend the tasks dictionary with new models for each new page
        tasks[key]["models"].extend(Model(**model) for model in sub_page_resp.models)

  0%|          | 0/36 [00:00<?, ?it/s]

requesting: https://huggingface.co/models-json?pipeline_tag=feature-extraction&sort=downloads


  0%|          | 0/173 [00:00<?, ?it/s]

requesting: https://huggingface.co/models-json?pipeline_tag=feature-extraction&p=1&sort=downloads
requesting: https://huggingface.co/models-json?pipeline_tag=feature-extraction&p=2&sort=downloads
requesting: https://huggingface.co/models-json?pipeline_tag=feature-extraction&p=3&sort=downloads
requesting: https://huggingface.co/models-json?pipeline_tag=feature-extraction&p=4&sort=downloads
requesting: https://huggingface.co/models-json?pipeline_tag=feature-extraction&p=5&sort=downloads
requesting: https://huggingface.co/models-json?pipeline_tag=feature-extraction&p=6&sort=downloads
requesting: https://huggingface.co/models-json?pipeline_tag=feature-extraction&p=7&sort=downloads
requesting: https://huggingface.co/models-json?pipeline_tag=feature-extraction&p=8&sort=downloads
requesting: https://huggingface.co/models-json?pipeline_tag=feature-extraction&p=9&sort=downloads
requesting: https://huggingface.co/models-json?pipeline_tag=feature-extraction&p=10&sort=downloads
requesting: https:/

  0%|          | 0/330 [00:00<?, ?it/s]

...

requesting: https://huggingface.co/models-json?pipeline_tag=tabular-regression&p=1&sort=downloads
requesting: https://huggingface.co/models-json?pipeline_tag=tabular-regression&p=2&sort=downloads
requesting: https://huggingface.co/models-json?pipeline_tag=tabular-regression&p=3&sort=downloads
requesting: https://huggingface.co/models-json?pipeline_tag=reinforcement-learning&sort=downloads


  0%|          | 0/963 [00:00<?, ?it/s]

requesting: https://huggingface.co/models-json?pipeline_tag=reinforcement-learning&p=1&sort=downloads
requesting: https://huggingface.co/models-json?pipeline_tag=reinforcement-learning&p=2&sort=downloads
requesting: https://huggingface.co/models-json?pipeline_tag=reinforcement-learning&p=3&sort=downloads
requesting: https://huggingface.co/models-json?pipeline_tag=reinforcement-learning&p=4&sort=downloads
requesting: https://huggingface.co/models-json?pipeline_tag=reinforcement-learning&p=5&sort=downloads
requesting: https://huggingface.co/models-json?pipeline_tag=reinforcement-learning&p=6&sort=downloads
requesting: https://huggingface.co/models-json?pipeline_tag=reinforcement-learning&p=7&sort=downloads
requesting: https://huggingface.co/models-json?pipeline_tag=reinforcement-learning&p=8&sort=downloads
requesting: https://huggingface.co/models-json?pipeline_tag=reinforcement-learning&p=9&sort=downloads
requesting: https://huggingface.co/models-json?pipeline_tag=reinforcement-learning

  0%|          | 0/1 [00:00<?, ?it/s]

requesting: https://huggingface.co/models-json?pipeline_tag=robotics&p=1&sort=downloads


In [18]:
# Convert pydantic models to dictionary for serialization
for key, task in tasks.items():
    tasks[key]['models'] = [model.model_dump() for model in tasks[key]['models']]

In [22]:
# Export tasks to json
with open('models.json', 'w') as f:
    json.dump(tasks, f, indent=2)

In [2]:
# Convert tasks to dataframe | Read from file if it exists
if pathlib.Path(os.getcwd(), "data", "models.csv").exists():
    df: pd.DataFrame =  pd.read_json(pathlib.Path(os.getcwd(), "data", "models.json"))
else: df: pd.DataFrame =  pd.DataFrame(tasks)

In [3]:
df.T.head()

,link,models
Feature Extraction,/models-json?pipeline_tag=feature-extraction,"[{'downloads': 3325267, 'id': 'google/vit-base..."
Text-to-Image,/models-json?pipeline_tag=text-to-image,"[{'downloads': 6232295, 'id': 'runwayml/stable..."
Image-to-Text,/models-json?pipeline_tag=image-to-text,"[{'downloads': 1917564, 'id': 'nlpconnect/vit-..."
Text-to-Video,/models-json?pipeline_tag=text-to-video,"[{'downloads': 36625, 'id': 'cerspense/zerosco..."
Visual Question Answering,/models-json?pipeline_tag=visual-question-answ...,"[{'downloads': 185024, 'id': 'google/pix2struc..."


In [4]:
# Create new dataframe using data in the models column
models_df = pd.DataFrame(df.T['models'].explode().tolist())

In [5]:
models_df

,downloads,id,lastModified,likes,pipeline_tag,private,repoType,author,authorData
0,3325267,google/vit-base-patch16-224-in21k,2023-02-27T15:04:22.000Z,61,feature-extraction,False,model,google,{'avatarUrl': 'https://aeiljuispo.cloudimg.io/...
1,1603214,YituTech/conv-bert-base,2021-02-24T11:26:14.000Z,4,feature-extraction,False,model,YituTech,{'avatarUrl': 'https://www.gravatar.com/avatar...
2,966618,facebook/bart-large,2022-06-03T10:00:20.000Z,113,feature-extraction,False,model,facebook,{'avatarUrl': 'https://aeiljuispo.cloudimg.io/...
3,805160,microsoft/wavlm-large,2022-02-02T21:21:50.000Z,36,feature-extraction,False,model,microsoft,{'avatarUrl': 'https://aeiljuispo.cloudimg.io/...
4,756029,indobenchmark/indobert-base-p1,2021-05-19T20:22:23.000Z,7,feature-extraction,False,model,indobenchmark,{'avatarUrl': 'https://aeiljuispo.cloudimg.io/...
...,...,...,...,...,...,...,...,...,...
180160,0,Bbyaz/robotics-deep-rl-grasping,2023-02-20T12:38:54.000Z,2,robotics,False,model,Bbyaz,{'avatarUrl': '/avatars/8311c9c8cebbff64b3cb5c...
180161,0,Beemarkethk/Beemarket,2023-04-27T14:30:38.000Z,1,robotics,False,model,Beemarkethk,{'avatarUrl': '/avatars/16ca7b5e02336e440ff83c...
180162,0,sirbrentmichaelskoda/Auto-GBT-Dream-Team-Model,2023-06-07T10:08:04.000Z,0,robotics,False,model,sirbrentmichaelskoda,{'avatarUrl': 'https://aeiljuispo.cloudimg.io/...
180163,0,Gage888/GageAI,2023-06-18T22:51:32.000Z,0,robotics,False,model,Gage888,{'avatarUrl': '/avatars/4be7b140197263aea29328...


In [6]:
# Export to csv
models_df.to_csv(pathlib.Path(os.getcwd(), "data", "models.csv"), sep=',', encoding='utf-8', header=True, index=False)